本例对比了三种水体提取算法，三种算法如下：

（1）基于mNDWI（Modified Normal Differential Water Index），其中mNDWI的阈值设为0.15。提取结果的layer name 是 water_mndwi

（2）基于NDVI（Normal Differential vegetation Index）、mNDWI（Modified Normal Differential Water Index）、EVI（enhanced vegetation index）的一种算法。原理：（mNDWI > NDVI and EVI < 0.1）OR （mNDWI > EVI and EVI < 0.1 ）。提取结果的layer name 是 water_ndvi_mndwi_evi

（3）基于NDVI（Normal Differential vegetation Index）、mNDWI（Modified Normal Differential Water Index）的一种算法。原理：mNDWI > NDVI。提取结果的layer name 是 water_ndvi_mndwi

（4）AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7

In [ ]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
roi = ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

In [ ]:
def NDVI(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    return image.addBands(mndwi)

def EVI(image):
    evi = image.expression('((nir - red) / (nir + 6*red - 7.5*blue + 1)) * 2.5 ',
        {
          'blue': image.select('B2'),   
          'red': image.select('B4'),
          'nir': image.select('B5'),
        }).rename('EVI')
    return image.addBands(evi)
# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def AWEIsh(image):
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)


# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

# 可视化参数设置
vis_params = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

In [ ]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2016-07-01', '2016-07-31') \
    .map(maskL8sr)

In [ ]:
landsat_map = landsat_images.map(NDVI).map(mNDWI).map(EVI)
wuhan_landsat = landsat_map.mosaic().clip(roi)
Map.addLayer(wuhan_landsat,vis_params,'wuhan landsat')

# 基于mNDWI的水体提取，阈值设为0.15

In [ ]:
histogram = wuhan_landsat.select('mNDWI').reduceRegion(**{
    'reducer': ee.Reducer.histogram(100).combine(ee.Reducer.mean(),'',True).combine(ee.Reducer.variance(),'',True),
    'geometry': roi.geometry(), 
    'scale': 30,
    'bestEffort': True
})

def sumSquare(i,histogram,counts,means,size,mean):
    aCounts = counts.slice(0, 0, i)
    aCountTotal = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCountTotal)
       
    bCounts = counts.slice(0, i, size)
    bCountTotal = bCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    bMeans = means.slice(0, i, size)
    bMean = bMeans.multiply(bCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(bCountTotal)

    return aCountTotal.multiply(aMean.subtract(mean).pow(2)).add(bCountTotal.multiply(bMean.subtract(mean).pow(2)))
# Return the DN that maximizes interclass variance in B5 (in the region).
def otsu(histogram):
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    mean = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(counts.reduce(ee.Reducer.sum(), [0]).get([0]))
    
    indices = ee.List.sequence(1,size)
    # Compute between sum of squares, where each mean partitions the data.
    bss = indices.map(lambda i : sumSquare(i,histogram,counts,means,size,mean))
    # Return the mean value corresponding to the maximum BSS.
    return means.sort(bss).get([-1])
dic= 'mNDWI' + "_" + 'histogram'
threshold = otsu(histogram.get(dic))
print('threshold : ', threshold.getInfo())

In [ ]:
water_mndwi = wuhan_landsat.select('mNDWI').gt(threshold).selfMask()

Map.addLayer(water_mndwi,{'palette': 'cyan'},'water_mndwi')

# 基于 NDVI、mNDWI、EVI的水体提取

In [ ]:
# 当mNDWI > NDVI and EVI < 0.1时(情况一)
image_filter1 = wuhan_landsat.select('mNDWI').gt(wuhan_landsat.select('NDVI')).And(wuhan_landsat.select('EVI').lt(0.1))
wuhan_water1 = image_filter1.selfMask()
Map.addLayer(wuhan_water1,{'palette': 'green'},'water filter1')
# mNDWI > EVI and EVI < 0.1 时（情况二）
image_filter2 = wuhan_landsat.select('mNDWI').gt(wuhan_landsat.select('EVI')).And(wuhan_landsat.select('EVI').lt(0.1))
wuhan_water2 = image_filter2.selfMask()
Map.addLayer(wuhan_water2,{'palette': 'green'},'water filter2')
# 情况一与情况二并集
water_union = ee.ImageCollection([image_filter1,image_filter2]).sum().gt(0).selfMask()
Map.addLayer(water_union,{ 'palette': ['red']},'water_union')
# # 情况一与情况二并集
# water_ndvi_mndwi_evi= wuhan_water1.select('mNDWI').Or(wuhan_water2.select('mNDWI'))
# Map.addLayer(water_ndvi_mndwi_evi,{'palette': 'green'},'water_ndvi_mndwi_evi')

# 基于 NDVI、mNDWI的水体提取

In [ ]:
# 当mNDWI > NDVI时
wuhan_water = wuhan_landsat.select('mNDWI').gt(wuhan_landsat.select('NDVI'))
water_ndvi_mndwi = wuhan_water.select('mNDWI').gt(0).selfMask()

Map.addLayer(water_ndvi_mndwi,{'palette': 'blue'},'water_ndvi_mndwi')

# 基于 AWEI 的水体提取

In [ ]:
water_awei = wuhan_landsat.select('AWEI').gt(0).selfMask()

Map.addLayer(water_awei,{'palette': 'yellow'},'water_awei')

In [ ]:
water_mndwi_rename = water_mndwi.select('mNDWI').rename('class')
water_ndvi_mndwi_rename = water_ndvi_mndwi.select('mNDWI').rename('class')
water_awei_rename = water_awei.select('AWEI').rename('class')
wuhan_water = ee.ImageCollection([water_mndwi_rename,water_ndvi_mndwi_rename,water_awei_rename]).sum().eq(3).selfMask()
Map.addLayer(wuhan_water,{'palette': '99d9ea'},'wuhan water')

In [ ]:
image_filter1 = wuhan_landsat.select('mNDWI').gt(wuhan_landsat.select('NDVI')).And(wuhan_landsat.select('EVI').lt(0.1))
wuhan_water1 = image_filter1.selfMask()
image_filter = wuhan_landsat.select('mNDWI').gt(wuhan_landsat.select('NDVI'))

Map.addLayer(image_filter,{'palette': 'blue'},'water')
Map.addLayer(wuhan_water1,{'palette': 'blue'},'water1')
Map.addLayer(image_filter1,{'min': 0, 'max': 1, 'palette': ['red', 'blue']},'image_filter1')

In [ ]:
image_filter2 = wuhan_landsat.select('mNDWI').gt(wuhan_landsat.select('EVI')).And(wuhan_landsat.select('EVI').lt(0.1))
wuhan_water2 = image_filter2.selfMask()

Map.addLayer(wuhan_water2,{'palette': 'yellow'},'water2')
Map.addLayer(image_filter2,{'min': 0, 'max': 1, 'palette': ['red', 'blue']},'image_filter2')

In [ ]:
wuhan_waterbody = wuhan_water.select('mNDWI').Or(wuhan_water2.select('mNDWI'))
Map.addLayer(wuhan_waterbody,{'palette': 'blue'},'wuhan_waterbody')

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee

region = [113.6, 29.9, 115.1, 31.4] 
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(water, region=region, vis_params=vis_params)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'water', fontsize=20)